In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from stockfish import Stockfish
import chess
import chess.engine
import math
import random

In [2]:
# data = pd.read_csv('Valgte punkter fra ensemble')
df = pd.read_csv(r"chessData.csv")

In [3]:
data = ['a7a6', 'a2a3', 'b7b6']

In [4]:
path=("stockfish-windows-x86-64\stockfish\stockfish-windows-x86-64.exe")

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\nunni\AppData\Local\Temp\ipykernel_16672\315037162.py:1: SyntaxWarning: invalid escape sequence '\s'
  path=("stockfish-windows-x86-64\stockfish\stockfish-windows-x86-64.exe")


In [5]:
fen=("rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2")

In [6]:
def stockfish_evaluation(board, time_limit = 0.01):
    engine = chess.engine.SimpleEngine.popen_uci(path)
    result = engine.analyse(board, chess.engine.Limit(time=time_limit))
    return result['score']

def evaluate_move(stockfish,fen,move):
    board = chess.Board(fen)
    board.push_uci(move)
    return stockfish_evaluation(board)

board = chess.Board(fen)
result = stockfish_evaluation(board)

stockfish = Stockfish(path)

In [7]:
score = []

for i in range(len(data)):
    move = data[i]
    fen = df['FEN'][i]
    if 'w' in fen:
        end = evaluate_move(stockfish, fen, move).white().score()
        score.append(end - int(df['Evaluation'][i]))
    else:
        end = evaluate_move(stockfish, fen, move).black().score()
        score.append(end - int(df['Evaluation'][i]))
        
print(score)

[-57, -51, -88]


In [8]:
def evaluate_list_white(stockfish,fen,list):
    res = []
    for move in list:
        res.append(evaluate_move(stockfish,fen,move).white().score())
    return res

def evaluate_list_black(stockfish,fen,list):
    res = []
    for move in list:
        res.append(evaluate_move(stockfish,fen,move).black().score())
    return res

def gen_legal_moves(board,frac=1):
    legal_moves = board.legal_moves
    legal_moves = [str(move) for move in legal_moves]
    if frac == 1:
        return legal_moves
    else:
        return np.random.choice(legal_moves, math.floor(frac*len(legal_moves)), replace=False)

In [12]:
mean_score = []
for i in range(len(data)):
    move = data[i]
    fen = df['FEN'][i]
    if 'w' in fen:
        move_score = evaluate_list_white(stockfish,fen,gen_legal_moves(chess.Board(fen))),"\n",gen_legal_moves(chess.Board(fen))
        for j in range(len(move_score[0])):
            mean_score.append(sum(move_score[0][j] - int(df['Evaluation'][i]))/len(range(len(move_score))))
    else:
        move_score = evaluate_list_black(stockfish,fen,gen_legal_moves(chess.Board(fen))),"\n",gen_legal_moves(chess.Board(fen))
        for j in range(len(move_score[0])):
            mean_score.append(sum(move_score[0][j] - int(df['Evaluation'][i]))/len(range(len(move_score))))
    
        
print(move_score)

<UciProtocol (pid=15192)>: stderr >> terminate called after throwing an instance of 'std::bad_alloc'
<UciProtocol (pid=15192)>: stderr >>   what():  std::bad_alloc


EngineTerminatedError: engine process died unexpectedly (exit code: 3)